In [1]:
from datasets import Dataset
import pandas as pd
import numpy as np
!pip install datasets --quiet
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import seaborn as sns

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertConfig, Trainer, TrainingArguments
import torch
from transformers.models.bert.modeling_bert import BertModel
from torch import nn
from transformers.trainer_utils import set_seed
import random
set_seed(42)

2024-06-14 01:23:10.610262: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 01:23:10.610405: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 01:23:10.751163: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train = pd.read_csv('/kaggle/input/cwi-dataset/cwi_training.txt', delimiter = "\t", header = None)
train.columns = ["sentence", "token", "index", "score"]

test = pd.read_csv('/kaggle/input/cwi-dataset/cwi_testing_annotated.txt', delimiter = "\t", header = None)
test.columns = ["sentence", "token", "index", "score"]

In [51]:
train

,sentence,token,index,score
0,In 1832 his family emigrated thence to Bellevi...,thence,5,1
1,In 1832 his family emigrated thence to Bellevi...,apprenticed,13,1
2,In 1832 his family emigrated thence to Bellevi...,family,3,0
3,In 1832 his family emigrated thence to Bellevi...,emigrated,4,0
4,In 1832 his family emigrated thence to Bellevi...,printer,16,0
...,...,...,...,...
2232,The short words math or maths are often used f...,basic,14,0
2233,The short words math or maths are often used f...,algebra,15,1
2234,The short words math or maths are often used f...,young,17,0
2235,The short words math or maths are often used f...,students,18,0


In [3]:
train.drop(["index"], axis = 1, inplace = True)

In [4]:
eval_data = test[:500]
test_data = test[500:]

test_data.dropna(inplace = True)
labels = test_data["score"]
test_data.drop(["score", "index"], axis = 1, inplace = True)

/tmp/ipykernel_34/2710407573.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.dropna(inplace = True)
/tmp/ipykernel_34/2710407573.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(["score", "index"], axis = 1, inplace = True)


In [5]:
 def remove_quotations(text):
    text = text.replace('"', '')
    text = text.replace("'", "")
    return text

def preprocess_function(examples):
    sentences = [remove_quotations(f"[CLS] {s} [SEP] {t}") for s, t in zip(examples['sentence'], examples['token'])] 
    tokenized_inputs = tokenizer(sentences, padding=True, truncation=True, max_length=64)
    tokenized_inputs['labels'] = examples['score']
    return tokenized_inputs

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test_data)
eval_dataset = Dataset.from_pandas(eval_data)

In [7]:
# Apply preprocessing to datasets
train_dataset = train_dataset.map(preprocess_function, batched=True, batch_size=512)
# test_dataset = test_dataset.map(preprocess_function, batched=True, batch_size=16)
eval_dataset = eval_dataset.map(preprocess_function, batched=True, batch_size=32)

Map:   0%|          | 0/2237 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [57]:
# NEW
from transformers import BertForSequenceClassification, BertConfig

# Define the model path
model_path = "prajjwal1/bert-medium"


# Configure the model for binary classification
config = BertConfig.from_pretrained(model_path, num_labels = 2, hidden_dropout_prob = 0.3, attention_probs_dropout_prob = 0.3,  output_hidden_states=False, output_attention=False)

# Load the model with the specified configuration
model = BertForSequenceClassification.from_pretrained(model_path, config=config)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-medium and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from sklearn.metrics import accuracy_score, recall_score
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification


def g_score(labels, predictions):
    accuracy = accuracy_score(labels, predictions)
    recall = recall_score(labels, predictions)
    if (accuracy + recall) == 0:
        return 0
    return (2 * accuracy * recall) / (accuracy + recall)

In [10]:
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)

    return {"g_score": g_score(labels, predictions)}

In [11]:
# Preprocess function that uses the tokenizer
def preprocess_test(examples):
    # Combining sentence and token with a special format and removing quotations
    sentences = [remove_quotations(f"[CLS] {s} [SEP] {t}") for s, t in zip(examples['sentence'], examples['token'])]
    # Tokenizing the sentences
    tokenized_inputs = tokenizer(sentences, padding=True, truncation=True, max_length=64)
    return tokenized_inputs

test_dataset = test_dataset.map(preprocess_test, batched=True, batch_size=512)

Map:   0%|          | 0/87719 [00:00<?, ? examples/s]

In [12]:
train_dataset

Dataset({
    features: ['sentence', 'token', 'score', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 2237
})

In [13]:
eval_dataset

Dataset({
    features: ['sentence', 'token', 'index', 'score', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 500
})

In [14]:
training_args = TrainingArguments(
    output_dir='/kaggle/working/model',
    report_to="none",
    do_train=True,
    do_eval=True,
    num_train_epochs=1,
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./logs',
    logging_steps=20,
    evaluation_strategy="steps",
    eval_steps=20,
    save_strategy="steps",
    fp16=torch.cuda.is_available(),  # Enables mixed precision training if a CUDA GPU is available
    load_best_model_at_end=True,
    lr_scheduler_type='linear', 
    learning_rate=4e-5,
    logging_first_step=True,
    metric_for_best_model = 'g_score',
    greater_is_better = True# Log the first step
)

# Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [74]:
trainer.train()

Step,Training Loss,Validation Loss,G Score
20,0.507000,0.910203,0.803478
40,0.471000,0.851614,0.802388
60,0.362400,0.922749,0.790214
80,0.364300,0.958621,0.774223
100,0.320700,0.715424,0.750638
120,0.256800,0.963711,0.756988
140,0.362900,0.761073,0.606083
160,0.527900,0.886031,0.727442
180,0.296200,0.864750,0.705526
200,0.435600,0.837712,0.684241


TrainOutput(global_step=280, training_loss=0.3991632980959756, metrics={'train_runtime': 23.0089, 'train_samples_per_second': 97.223, 'train_steps_per_second': 12.169, 'total_flos': 19987283858292.0, 'train_loss': 0.3991632980959756, 'epoch': 1.0})

In [25]:
predictions = trainer.predict(test_dataset)

In [26]:
def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

probabilities = softmax(predictions[0])

# Convert probabilities to binary class predictions (0 or 1)
predictions = np.argmax(probabilities, axis=1)

print("Probabilities:\n", probabilities)
print("Predictions (0 or 1):\n", predictions)

Probabilities:
 [[9.9649614e-01 3.5038227e-03]
 [9.9685800e-01 3.1419473e-03]
 [3.3242542e-02 9.6675748e-01]
 ...
 [8.2474656e-04 9.9917525e-01]
 [9.9845731e-01 1.5427324e-03]
 [1.0044163e-03 9.9899560e-01]]
Predictions (0 or 1):
 [0 0 1 ... 1 0 1]


In [78]:
g_score(labels, predictions) * 100

75.02678241589383